In [8]:
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.inizalizers import RandomNormal, RandomUniform
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from keras.regularizers import l1, l2
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
import keras_tuner
import keras
from keras import layers
import itertools
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

seed = 7
tf.random.set_seed(seed)

In [2]:
def load_monks(path):
    train = pd.read_csv(f"{path}.train", header=None, sep=" ")
    test = pd.read_csv(f"{path}.test", header=None, sep=" ")
    
    train.drop(0, axis=1, inplace=True)
    test.drop(0, axis=1, inplace=True)
    train.drop(8, axis=1, inplace=True)
    test.drop(8, axis=1, inplace=True)
    
    y_train = train.iloc[:, 0].to_numpy().astype(np.float64)
    x_train = train.iloc[:, 1:].to_numpy().astype(np.float64)
    y_test = test.iloc[:, 0].to_numpy().astype(np.float64)
    x_test = test.iloc[:, 1:].to_numpy().astype(np.float64)
    
    return x_train, y_train, x_test, y_test
    

def load_cup():
    train = pd.read_csv("../data/ML-CUP22-TR.csv", header=None, sep=",")
    test = pd.read_csv("../data/ML-CUP22-TS.csv", header=None, sep=",")
    train.drop(0, axis=1, inplace=True)
    test.drop(0, axis=1, inplace=True)
    train.head()
    y_train = train.iloc[:, :2].to_numpy().astype(np.float64)
    x_train = train.iloc[:, 2:].to_numpy().astype(np.float64)
    x_test = test.to_numpy().astype(np.float64)
    return x_train, y_train, x_test, None
    
def load_dataset(dataset):
    assert dataset in ["monks1", "monks2", "monks3", "cup"]
    
    if dataset == "monks1":
        return load_monks("./../data/monks-1")
    elif dataset == "monks2":
        return load_monks("./../data/monks-2")
    elif dataset == "monks3":
        return load_monks("./../data/monks-3")
    else:
        return load_cup()

In [3]:
x_train, y_train, x_test, y_test = load_dataset("monks1")
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(124, 6) (124,)
(432, 6) (432,)


In [11]:
params = {
    "learning_rate": [0.001, 0.01, 0.1, 1],
    "batch_size": [16 , 64],
    "momentum": [0.1, 0.5, 0.9],
    "regularizer": [0.01, 0.1, 1],
    "layers": [
        {
            "activations": ["relu"],
            "units": [1,2,3,4,5,6,7,8],
        },
    ]
}

k_fold_val = {
    "type": "k-fold",
    "n_folds": [2, 3, 5, 10, 15, 20]
}

hold_out_val = {
    "type": "hold-out",
    "val_split": 0.2
}


def merge_dicts(dicts):
    merged = {}
    for d in dicts:
        merged = {**merged, **d}
    return merged



def get_params_configurations(params):
    
    keys = list(params.keys())
    keys.remove("layers")
    sorted_keys = sorted(keys)
    arguments = []
    for key in sorted_keys:
        arguments.append(params[key])
    
    arguments = tuple(arguments)
    all_params = list(itertools.product(*arguments))
    
    configurations = []
    
    for conf in all_params:
        
        dict_conf = {}
        
        for i in range(len(sorted_keys)):
            dict_conf[sorted_keys[i]] = conf[i]
        
        configurations.append(dict_conf)
        
    return configurations



def get_layers_configurations(params, configurations):
    layers = params["layers"]
    layers_params_confs = []
    
    for n_layer in range(len(layers)):
        layers_params_confs.append([]) 
    
    for i in range(len(layers)):
        layer = layers[i]
        activations = layer["activations"]
        units = layer["units"]
        layer_confs = list(itertools.product(activations, units))
        for conf in layer_confs:
            conf_dict = {
                f"activations_{i+1}": conf[0],
                f"units_{i+1}": conf[1]
            }
            layers_params_confs[i].append(conf_dict)
        
    layers_confs = []
    for i in range(len(layers_params_confs)):
        confs = layers_params_confs[:i+1]
        confs = tuple(confs)
        confs = list(itertools.product(*confs))
        for conf in confs:
            layers_confs.append(merge_dicts(conf))
                
    return layers_confs



def get_configurations(params):
    configurations = get_params_configurations(params)
    layers_confs = get_layers_configurations(params, configurations)
    configurations = list(itertools.product(configurations, layers_confs))
    configurations_merged = []
    
    for conf in configurations:
        configurations_merged.append(merge_dicts(conf))
        
    
    return configurations_merged
    
    
def get_model(config):    
    lr = config["learning_rate"]
    reg = config["regularizer"]
    momentum = config["momentum"]
    
    n_layers = 0
    for key in config.keys():
        if key.startswith("units_"):
            n_layers += 1

    model = Sequential()
    model.add(Dense(6, input_dim=6, activation="relu", kernel_regularizer=l2(reg), bias_regularizer=l2(reg)))
    
    for l in range(1,n_layers+1):
        activation = config[f"activations_{l}"]
        n_units = config[f"units_{l}"]
        model.add(Dense(n_units, activation=activation, kernel_regularizer=l2(reg), bias_regularizer=l2(reg)))
        
    model.add(Dense(1, activation="sigmoid", kernel_regularizer=l2(reg), bias_regularizer=l2(reg)))
    
    optimizer = SGD(learning_rate=lr, momentum=momentum)
    
    model.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'])
    return model
    
    
def fit_model(model, x_train, y_train, max_epochs, batch_size, val_split):
    
    early_stop = EarlyStopping(
        monitor="val_loss",
        min_delta=0.001,
        patience=10,
        verbose=0,
        mode="auto",
        restore_best_weights=True,
    )
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_split, shuffle=True)
    
    history = model.fit(x_train, y_train, epochs=max_epochs, batch_size=batch_size,
                        verbose=0, validation_data=(x_val, y_val), callbacks=[early_stop])
    
    train_loss = history.history["loss"]
    train_acc = history.history["accuracy"]
    
    val_loss = history.history["val_loss"]
    val_acc = history.history["val_accuracy"]
    
    print(len(train_loss), val_acc[-1])
    
    
def model_selection(configurations, x_train, y_train):
    with tqdm(total=len(configurations)) as pbar:
        for config in configurations:
            print(config)
            model = get_model(config)
            fit_model(model, x_train, y_train, 500, config["batch_size"], 0.15)
            pbar.update(1)
    
    

def grid_search(params, x_train, y_train, validation=None):
    configurations = get_configurations(params)
    model_selection(configurations, x_train, y_train)        
        
        

grid_search(params, x_train, y_train)

  0%|                                                   | 0/576 [00:00<?, ?it/s]

{'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.1, 'regularizer': 0.01, 'activations_1': 'relu', 'units_1': 1}


  0%|                                           | 1/576 [00:00<09:27,  1.01it/s]

11 0.3684210479259491
{'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.1, 'regularizer': 0.01, 'activations_1': 'relu', 'units_1': 2}


  0%|▏                                          | 2/576 [00:09<49:28,  5.17s/it]

...

202 0.5789473652839661
{'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.1, 'regularizer': 0.01, 'activations_1': 'relu', 'units_1': 8}


  1%|▌                                        | 8/576 [01:03<1:16:48,  8.11s/it]

21 0.6315789222717285
{'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.1, 'regularizer': 0.1, 'activations_1': 'relu', 'units_1': 1}


  1%|▌                                        | 8/576 [01:08<1:21:11,  8.58s/it]


KeyboardInterrupt: 

In [20]:
def build_model():
    model = Sequential()
    model.add(Dense(5, input_dim=6, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    optimizer = SGD(learning_rate=0.1, momentum=0.9)
    model.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'])
    return model

In [21]:
model = build_model()

early_stop = EarlyStopping(
        monitor="val_loss",
        min_delta=0,
        patience=100,
        verbose=0,
        mode="auto",
        restore_best_weights=True,
    )


x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, shuffle=True)

model.fit(x_train, y_train, epochs=300, batch_size=16, callbacks=[early_stop], validation_data=(x_val, y_val))

Epoch 1/300
4/4 [==============================] - 0s 49ms/step - loss: 0.2291 - accuracy: 0.6415 - val_loss: 0.1571 - val_accuracy: 0.8000
Epoch 2/300
4/4 [==============================] - 0s 13ms/step - loss: 0.2019 - accuracy: 0.6981 - val_loss: 0.1886 - val_accuracy: 0.5000
Epoch 3/300
4/4 [==============================] - 0s 11ms/step - loss: 0.1973 - accuracy: 0.7547 - val_loss: 0.2832 - val_accuracy: 0.5000
Epoch 4/300
4/4 [==============================] - 0s 12ms/step - loss: 0.1509 - accuracy: 0.8491 - val_loss: 0.1432 - val_accuracy: 0.8000
Epoch 5/300
4/4 [==============================] - 0s 13ms/step - loss: 0.2094 - accuracy: 0.6792 - val_loss: 0.1227 - val_accuracy: 0.9000
Epoch 6/300
4/4 [==============================] - 0s 11ms/step - loss: 0.1523 - accuracy: 0.7925 - val_loss: 0.1926 - val_accuracy: 0.6000
Epoch 7/300
4/4 [==============================] - 0s 10ms/step - loss: 0.1462 - accuracy: 0.8302 - val_loss: 0.1295 - val_accuracy: 0.8000
Epoch 8/300
4/4 [===

4/4 [==============================] - 0s 10ms/step - loss: 0.0942 - accuracy: 0.9057 - val_loss: 0.2824 - val_accuracy: 0.6000
Epoch 60/300
4/4 [==============================] - 0s 9ms/step - loss: 0.0934 - accuracy: 0.9057 - val_loss: 0.2068 - val_accuracy: 0.8000
Epoch 61/300
4/4 [==============================] - 0s 8ms/step - loss: 0.0965 - accuracy: 0.8868 - val_loss: 0.2033 - val_accuracy: 0.8000
Epoch 62/300
4/4 [==============================] - 0s 13ms/step - loss: 0.0974 - accuracy: 0.9057 - val_loss: 0.2342 - val_accuracy: 0.6000
Epoch 63/300
4/4 [==============================] - 0s 13ms/step - loss: 0.1032 - accuracy: 0.8868 - val_loss: 0.2343 - val_accuracy: 0.6000
Epoch 64/300
4/4 [==============================] - 0s 12ms/step - loss: 0.0947 - accuracy: 0.9057 - val_loss: 0.1786 - val_accuracy: 0.8000
Epoch 65/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0885 - accuracy: 0.9057 - val_loss: 0.2373 - val_accuracy: 0.6000
Epoch 66/300
4/4 [==========

4/4 [==============================] - 0s 11ms/step - loss: 0.0560 - accuracy: 0.9245 - val_loss: 0.1735 - val_accuracy: 0.7000
Epoch 118/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0555 - accuracy: 0.9434 - val_loss: 0.1701 - val_accuracy: 0.7000
Epoch 119/300
4/4 [==============================] - 0s 12ms/step - loss: 0.0566 - accuracy: 0.9245 - val_loss: 0.1635 - val_accuracy: 0.8000
Epoch 120/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0505 - accuracy: 0.9245 - val_loss: 0.1482 - val_accuracy: 0.9000
Epoch 121/300
4/4 [==============================] - 0s 12ms/step - loss: 0.0537 - accuracy: 0.9623 - val_loss: 0.1758 - val_accuracy: 0.8000
Epoch 122/300
4/4 [==============================] - 0s 12ms/step - loss: 0.0617 - accuracy: 0.9057 - val_loss: 0.1953 - val_accuracy: 0.8000
Epoch 123/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0476 - accuracy: 0.9434 - val_loss: 0.1464 - val_accuracy: 0.8000
Epoch 124/300
4/4 [=

Epoch 175/300
4/4 [==============================] - 0s 13ms/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.1142 - val_accuracy: 0.9000
Epoch 176/300
4/4 [==============================] - 0s 20ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.1086 - val_accuracy: 0.9000
Epoch 177/300
4/4 [==============================] - 0s 23ms/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.1049 - val_accuracy: 0.9000
Epoch 178/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.1133 - val_accuracy: 0.9000
Epoch 179/300
4/4 [==============================] - 0s 11ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.1161 - val_accuracy: 0.9000
Epoch 180/300
4/4 [==============================] - 0s 14ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.1115 - val_accuracy: 0.9000
Epoch 181/300
4/4 [==============================] - 0s 12ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.1068 - val_accuracy: 0.9000
Epoch 

In [24]:
model.predict(x_test)

14/14 [==============================] - 0s 1ms/step


array([[0.22917773],
       [0.2229525 ],
       [0.26184192],
       [0.25502297],
       [0.2973657 ],
       [0.28998503],
       [0.33551827],
       [0.32763222],
       [0.21924704],
       [0.21321695],
       [0.25095665],
       [0.24432737],
       [0.28557494],
       [0.27837104],
       [0.32291   ],
       [0.31517974],
       [0.20962965],
       [0.20379503],
       [0.24037653],
       [0.23393947],
       [0.27406937],
       [0.26704723],
       [0.3105541 ],
       [0.30298725],
       [0.23476976],
       [0.22843766],
       [0.2679539 ],
       [0.2610321 ],
       [0.3039654 ],
       [0.2964901 ],
       [0.34255195],
       [0.33458385],
       [0.2246674 ],
       [0.21852994],
       [0.25690302],
       [0.2501691 ],
       [0.29202175],
       [0.28472003],
       [0.32981056],
       [0.32199368],
       [0.21487768],
       [0.2089356 ],
       [0.24615462],
       [0.23961158],
       [0.28035837],
       [0.2732357 ],
       [0.31731436],
       [0.309